In [ ]:
# Cell 1: Import all required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from faker import Faker
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Set up plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("✅ All libraries imported successfully!")

In [ ]:
# Cell 2: Traffic Data Generator Class
class TrafficDataGenerator:
    """
    A class to generate synthetic traffic data for urban traffic analysis.
    """

    def __init__(self, locale='en_US'):
        """
        Initialize the TrafficDataGenerator.

        Args:
            locale (str): Locale for Faker data generation
        """
        self.fake = Faker(locale)
        np.random.seed(42)

    def generate_traffic_data(self, n_records=1000):
        """
        Generate synthetic traffic data records.

        Args:
            n_records (int): Number of records to generate

        Returns:
            pandas.DataFrame: DataFrame containing traffic data
        """
        print(f"🚗 Generating {n_records} traffic data records...")

        data = []
        base_date = datetime(2024, 1, 1)

        for i in range(n_records):
            record = self._generate_single_record(i, base_date)
            data.append(record)

        df = pd.DataFrame(data)
        df['timestamp'] = pd.to_datetime(df['timestamp'])
        return df

    def _generate_single_record(self, record_id, base_date):
        """
        Generate a single traffic data record.
        """
        congestion_level = np.random.choice(
            ['Light', 'Medium', 'Heavy', 'Very Heavy'],
            p=[0.3, 0.4, 0.2, 0.1]
        )

        # Generate speed based on congestion level
        if congestion_level == 'Light':
            speed = np.random.normal(60, 10)
        elif congestion_level == 'Medium':
            speed = np.random.normal(40, 8)
        elif congestion_level == 'Heavy':
            speed = np.random.normal(20, 5)
        else:  # Very Heavy
            speed = np.random.normal(10, 3)

        record = {
            'id': record_id + 1,
            'timestamp': base_date + timedelta(hours=record_id),
            'location_name': self.fake.street_name(),
            'latitude': np.random.uniform(35.6, 35.8),
            'longitude': np.random.uniform(51.2, 51.5),
            'vehicle_count': np.random.poisson(50),
            'average_speed': max(5, speed),
            'congestion_level': congestion_level,
            'road_type': np.random.choice(
                ['Arterial', 'Secondary', 'Primary', 'Expressway'],
                p=[0.4, 0.3, 0.2, 0.1]
            ),
            'weather': np.random.choice(
                ['Sunny', 'Cloudy', 'Rainy', 'Snowy'],
                p=[0.6, 0.2, 0.15, 0.05]
            )
        }

        return record

# Generate sample data
generator = TrafficDataGenerator()
traffic_data = generator.generate_traffic_data(1500)
print("✅ Traffic data generated successfully!")
print(f"📊 Dataset shape: {traffic_data.shape}")

In [ ]:
# Cell 3: Display basic information about the dataset
print("🔍 Dataset Overview:")
print("=" * 50)
print(traffic_data.info())
print("\n📈 First 5 rows:")
print(traffic_data.head())
print("\n📊 Basic Statistics:")
print(traffic_data.describe())

'Miller Gardens'

In [ ]:
# Cell 4: Data Processor Class
class TrafficDataProcessor:
    """
    A class to process and analyze traffic data.
    """

    def __init__(self, data):
        self.data = data
        self.analysis_results = {}

    def add_calculated_features(self):
        """
        Add calculated features to the traffic data.
        """
        print("🧮 Adding calculated features...")

        # Calculate traffic index
        self.data['traffic_index'] = self._calculate_traffic_index()

        # Add time-based features
        self._add_time_features()

        # Calculate traffic efficiency
        self.data['efficiency'] = self._calculate_efficiency()

        return self.data

    def _calculate_traffic_index(self):
        """Calculate traffic index based on speed and vehicle count."""
        normalized_speed = (100 - self.data['average_speed']) / 100
        normalized_volume = self.data['vehicle_count'] / 100
        return (normalized_speed * normalized_volume) * 100

    def _add_time_features(self):
        """Add time-based features to the data."""
        self.data['hour'] = self.data['timestamp'].dt.hour
        self.data['day_of_week'] = self.data['timestamp'].dt.day_name()
        self.data['time_of_day'] = self.data['hour'].apply(self._categorize_time_of_day)

    def _categorize_time_of_day(self, hour):
        """Categorize hours into time of day segments."""
        if 6 <= hour < 10:
            return 'Morning'
        elif 10 <= hour < 14:
            return 'Noon'
        elif 14 <= hour < 18:
            return 'Afternoon'
        elif 18 <= hour < 22:
            return 'Evening'
        else:
            return 'Midnight'

    def _calculate_efficiency(self):
        """Calculate traffic efficiency metric."""
        return np.where(
            self.data['average_speed'] > 20,
            self.data['average_speed'] / np.maximum(self.data['vehicle_count'], 1),
            0
        )

    def analyze_traffic_patterns(self):
        """Analyze traffic patterns from the data."""
        print("📊 Analyzing traffic patterns...")

        analysis = {}

        # Hourly analysis
        analysis['hourly'] = self._analyze_hourly_patterns()

        # Daily analysis
        analysis['daily'] = self._analyze_daily_patterns()

        # Road type analysis
        analysis['road_type'] = self._analyze_road_type_patterns()

        # Weather impact analysis
        analysis['weather'] = self._analyze_weather_impact()

        self.analysis_results = analysis
        return analysis

    def _analyze_hourly_patterns(self):
        """Analyze hourly traffic patterns."""
        return self.data.groupby('hour').agg({
            'vehicle_count': ['mean', 'std'],
            'average_speed': ['mean', 'std'],
            'traffic_index': 'mean'
        }).round(2)

    def _analyze_daily_patterns(self):
        """Analyze daily traffic patterns."""
        weekday_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
        daily_stats = self.data.groupby('day_of_week').agg({
            'vehicle_count': 'mean',
            'average_speed': 'mean',
            'traffic_index': 'mean'
        })
        return daily_stats.reindex(weekday_order)

    def _analyze_road_type_patterns(self):
        """Analyze patterns by road type."""
        return self.data.groupby('road_type').agg({
            'vehicle_count': 'mean',
            'average_speed': 'mean',
            'traffic_index': 'mean',
            'efficiency': 'mean'
        }).round(2)

    def _analyze_weather_impact(self):
        """Analyze weather impact on traffic."""
        return self.data.groupby('weather').agg({
            'vehicle_count': 'mean',
            'average_speed': 'mean',
            'traffic_index': 'mean'
        }).round(2)

    def get_summary_statistics(self):
        """Get summary statistics for the data."""
        return {
            'total_records': len(self.data),
            'avg_vehicle_count': self.data['vehicle_count'].mean(),
            'avg_speed': self.data['average_speed'].mean(),
            'avg_traffic_index': self.data['traffic_index'].mean(),
            'most_common_congestion': self.data['congestion_level'].mode()[0],
            'date_range': {
                'start': self.data['timestamp'].min(),
                'end': self.data['timestamp'].max()
            }
        }

# Process the data
processor = TrafficDataProcessor(traffic_data)
processed_data = processor.add_calculated_features()
analysis_results = processor.analyze_traffic_patterns()
summary_stats = processor.get_summary_statistics()

print("✅ Data processing completed!")

In [ ]:
# Cell 5: Display Summary Statistics
print("📈 TRAFFIC ANALYSIS SUMMARY")
print("=" * 40)
print(f"Total Records: {summary_stats['total_records']:,}")
print(f"Average Vehicle Count: {summary_stats['avg_vehicle_count']:.1f}")
print(f"Average Speed: {summary_stats['avg_speed']:.1f} km/h")
print(f"Average Traffic Index: {summary_stats['avg_traffic_index']:.2f}")
print(f"Most Common Congestion: {summary_stats['most_common_congestion']}")
print(f"Date Range: {summary_stats['date_range']['start'].strftime('%Y-%m-%d')} to {summary_stats['date_range']['end'].strftime('%Y-%m-%d')}")

In [ ]:
# Cell 6: Traffic Visualizer Class
class TrafficVisualizer:
    """
    A class for creating traffic data visualizations.
    """

    def __init__(self, data, analysis_results):
        self.data = data
        self.analysis_results = analysis_results
        self._setup_plot_style()

    def _setup_plot_style(self):
        """Set up consistent plot styling."""
        plt.rcParams['font.size'] = 12
        plt.rcParams['figure.figsize'] = (12, 8)
        sns.set_style("whitegrid")

    def create_main_dashboard(self):
        """Create the main dashboard with multiple subplots."""
        fig, axes = plt.subplots(2, 2, figsize=(20, 15))
        fig.suptitle('Comprehensive Urban Traffic Pattern Analysis', fontsize=20, fontweight='bold', y=0.95)

        self._plot_hourly_traffic(axes[0, 0])
        self._plot_daily_patterns(axes[0, 1])
        self._plot_traffic_heatmap(axes[1, 0])
        self._plot_congestion_distribution(axes[1, 1])

        plt.tight_layout()
        plt.show()

    def _plot_hourly_traffic(self, ax):
        """Plot hourly traffic patterns."""
        hourly_data = self.analysis_results['hourly']
        hours = hourly_data.index

        ax.plot(hours, hourly_data[('vehicle_count', 'mean')],
               marker='o', linewidth=3, markersize=6, label='Vehicle Count', color='blue', alpha=0.7)
        ax.set_title('Average Vehicle Count by Hour', fontsize=14, fontweight='bold')
        ax.set_xlabel('Hour of Day', fontsize=12)
        ax.set_ylabel('Vehicle Count', fontsize=12)
        ax.grid(True, alpha=0.3)
        ax.legend(fontsize=10)
        ax.set_xticks(range(0, 24, 2))
        ax.fill_between(hours, hourly_data[('vehicle_count', 'mean')], alpha=0.2, color='blue')

    def _plot_daily_patterns(self, ax):
        """Plot daily traffic patterns."""
        days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
        daily_data = self.analysis_results['daily']

        x_pos = np.arange(len(days))
        colors = plt.cm.Set3(np.linspace(0, 1, len(days)))
        bars = ax.bar(x_pos, daily_data['average_speed'], color=colors, alpha=0.8)

        ax.set_title('Average Speed by Day of Week', fontsize=14, fontweight='bold')
        ax.set_xlabel('Day of Week', fontsize=12)
        ax.set_ylabel('Average Speed (km/h)', fontsize=12)
        ax.set_xticks(x_pos)
        ax.set_xticklabels(days, rotation=45)

        # Add value labels on bars
        for bar, value in zip(bars, daily_data['average_speed']):
            ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5,
                   f'{value:.1f}', ha='center', va='bottom', fontweight='bold', fontsize=10)

    def _plot_traffic_heatmap(self, ax):
        """Create a heatmap of traffic distribution."""
        scatter = ax.scatter(self.data['longitude'], self.data['latitude'],
                           c=self.data['traffic_index'], cmap='Reds',
                           s=50, alpha=0.7, edgecolors='black', linewidth=0.5)

        ax.set_title('Traffic Heatmap - Urban Area', fontsize=14, fontweight='bold')
        ax.set_xlabel('Longitude', fontsize=12)
        ax.set_ylabel('Latitude', fontsize=12)
        plt.colorbar(scatter, ax=ax, label='Traffic Index')

        # Add grid
        ax.grid(True, alpha=0.3)

    def _plot_congestion_distribution(self, ax):
        """Plot congestion level distribution."""
        congestion_counts = self.data['congestion_level'].value_counts()
        colors = ['lightgreen', 'gold', 'orange', 'red']

        wedges, texts, autotexts = ax.pie(congestion_counts.values,
                                         labels=congestion_counts.index,
                                         autopct='%1.1f%%',
                                         startangle=90,
                                         colors=colors,
                                         shadow=True,
                                         explode=(0.05, 0.05, 0.05, 0.05))

        # Enhance text appearance
        for autotext in autotexts:
            autotext.set_color('white')
            autotext.set_fontweight('bold')
            autotext.set_fontsize(10)

        ax.set_title('Traffic Congestion Distribution', fontsize=14, fontweight='bold')

    def create_supplementary_charts(self):
        """Create supplementary analysis charts."""
        fig, axes = plt.subplots(2, 2, figsize=(20, 12))

        self._plot_speed_by_road_type(axes[0, 0])
        self._plot_density_vs_speed(axes[0, 1])
        self._plot_weather_impact(axes[1, 0])
        self._plot_efficiency_analysis(axes[1, 1])

        plt.tight_layout()
        plt.show()

    def _plot_speed_by_road_type(self, ax):
        """Plot speed distribution by road type."""
        road_types = self.data['road_type'].unique()
        speed_data = [self.data[self.data['road_type'] == road]['average_speed']
                     for road in road_types]

        box_plot = ax.boxplot(speed_data, labels=road_types, patch_artist=True)

        # Color the boxes
        colors = ['lightblue', 'lightgreen', 'lightcoral', 'lightsalmon']
        for patch, color in zip(box_plot['boxes'], colors):
            patch.set_facecolor(color)

        ax.set_title('Speed Distribution by Road Type', fontsize=14, fontweight='bold')
        ax.set_ylabel('Speed (km/h)', fontsize=12)
        ax.tick_params(axis='x', rotation=45)
        ax.grid(True, alpha=0.3)

    def _plot_density_vs_speed(self, ax):
        """Plot vehicle density vs speed relationship."""
        scatter = ax.scatter(self.data['vehicle_count'], self.data['average_speed'],
                           c=self.data['traffic_index'], cmap='viridis',
                           alpha=0.6, s=30)

        ax.set_title('Vehicle Density vs Average Speed', fontsize=14, fontweight='bold')
        ax.set_xlabel('Vehicle Count', fontsize=12)
        ax.set_ylabel('Average Speed (km/h)', fontsize=12)
        plt.colorbar(scatter, ax=ax, label='Traffic Index')

        # Add trend line
        z = np.polyfit(self.data['vehicle_count'], self.data['average_speed'], 1)
        p = np.poly1d(z)
        ax.plot(self.data['vehicle_count'], p(self.data['vehicle_count']),
               "r--", alpha=0.8, linewidth=2, label='Trend Line')
        ax.legend()

    def _plot_weather_impact(self, ax):
        """Plot weather impact on traffic."""
        weather_data = self.analysis_results['weather']
        x_pos = np.arange(len(weather_data.index))

        bars = ax.bar(x_pos, weather_data['average_speed'],
                     color=plt.cm.Pastel1(np.linspace(0, 1, len(weather_data))))

        ax.set_title('Weather Impact on Average Speed', fontsize=14, fontweight='bold')
        ax.set_xlabel('Weather Condition', fontsize=12)
        ax.set_ylabel('Average Speed (km/h)', fontsize=12)
        ax.set_xticks(x_pos)
        ax.set_xticklabels(weather_data.index)

        for bar, value in zip(bars, weather_data['average_speed']):
            ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5,
                   f'{value:.1f}', ha='center', va='bottom', fontweight='bold')

    def _plot_efficiency_analysis(self, ax):
        """Plot traffic efficiency analysis."""
        efficiency_by_road = self.data.groupby('road_type')['efficiency'].mean().sort_values(ascending=False)

        bars = ax.bar(range(len(efficiency_by_road)), efficiency_by_road.values,
                     color=plt.cm.Set2(np.linspace(0, 1, len(efficiency_by_road))))

        ax.set_title('Traffic Efficiency by Road Type', fontsize=14, fontweight='bold')
        ax.set_xlabel('Road Type', fontsize=12)
        ax.set_ylabel('Efficiency Score', fontsize=12)
        ax.set_xticks(range(len(efficiency_by_road)))
        ax.set_xticklabels(efficiency_by_road.index, rotation=45)

        for bar, value in zip(bars, efficiency_by_road.values):
            ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                   f'{value:.2f}', ha='center', va='bottom', fontweight='bold')

# Create visualizations
visualizer = TrafficVisualizer(processed_data, analysis_results)
print("🎨 Creating main dashboard...")

In [ ]:
# Cell 7: Display Main Dashboard
visualizer.create_main_dashboard()

In [ ]:
# Cell 8: Display Supplementary Charts
print("📊 Creating supplementary analysis charts...")
visualizer.create_supplementary_charts()

In [ ]:
# Cell 9: Advanced Time Series Analysis
print("⏰ Performing time series analysis...")

# Create time-based aggregations
hourly_aggregated = processed_data.groupby('hour').agg({
    'vehicle_count': 'mean',
    'average_speed': 'mean',
    'traffic_index': 'mean'
}).reset_index()

# Plot time series analysis
fig, axes = plt.subplots(2, 2, figsize=(20, 12))

# Hourly patterns with confidence intervals
hourly_stats = processed_data.groupby('hour')['vehicle_count'].agg(['mean', 'std', 'count'])
hours = hourly_stats.index

axes[0, 0].plot(hours, hourly_stats['mean'], linewidth=3, color='blue', label='Mean Vehicle Count')
axes[0, 0].fill_between(hours,
                       hourly_stats['mean'] - hourly_stats['std'],
                       hourly_stats['mean'] + hourly_stats['std'],
                       alpha=0.3, color='blue', label='±1 Std Dev')
axes[0, 0].set_title('Hourly Vehicle Count with Confidence Intervals', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('Hour of Day')
axes[0, 0].set_ylabel('Vehicle Count')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Correlation heatmap
numeric_columns = ['vehicle_count', 'average_speed', 'traffic_index', 'efficiency', 'hour']
correlation_matrix = processed_data[numeric_columns].corr()

im = axes[0, 1].imshow(correlation_matrix, cmap='coolwarm', aspect='auto', vmin=-1, vmax=1)
axes[0, 1].set_xticks(range(len(correlation_matrix.columns)))
axes[0, 1].set_yticks(range(len(correlation_matrix.columns)))
axes[0, 1].set_xticklabels(correlation_matrix.columns, rotation=45)
axes[0, 1].set_yticklabels(correlation_matrix.columns)
axes[0, 1].set_title('Feature Correlation Matrix', fontsize=14, fontweight='bold')

# Add correlation values to heatmap
for i in range(len(correlation_matrix.columns)):
    for j in range(len(correlation_matrix.columns)):
        text = axes[0, 1].text(j, i, f'{correlation_matrix.iloc[i, j]:.2f}',
                              ha="center", va="center", color="black", fontweight='bold')

plt.colorbar(im, ax=axes[0, 1])

# Traffic patterns by time of day
time_of_day_stats = processed_data.groupby('time_of_day').agg({
    'vehicle_count': 'mean',
    'average_speed': 'mean'
}).reindex(['Morning', 'Noon', 'Afternoon', 'Evening', 'Midnight'])

x_pos = np.arange(len(time_of_day_stats))
width = 0.35

axes[1, 0].bar(x_pos - width/2, time_of_day_stats['vehicle_count'], width,
               label='Vehicle Count', alpha=0.7, color='skyblue')
axes[1, 0].bar(x_pos + width/2, time_of_day_stats['average_speed'], width,
               label='Average Speed', alpha=0.7, color='lightcoral')

axes[1, 0].set_title('Traffic Patterns by Time of Day', fontsize=14, fontweight='bold')
axes[1, 0].set_xlabel('Time of Day')
axes[1, 0].set_ylabel('Values')
axes[1, 0].set_xticks(x_pos)
axes[1, 0].set_xticklabels(time_of_day_stats.index)
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Congestion level by road type
congestion_by_road = pd.crosstab(processed_data['road_type'], processed_data['congestion_level'], normalize='index') * 100

congestion_by_road.plot(kind='bar', stacked=True, ax=axes[1, 1],
                       color=['lightgreen', 'gold', 'orange', 'red'])
axes[1, 1].set_title('Congestion Level Distribution by Road Type', fontsize=14, fontweight='bold')
axes[1, 1].set_xlabel('Road Type')
axes[1, 1].set_ylabel('Percentage (%)')
axes[1, 1].legend(title='Congestion Level')
axes[1, 1].tick_params(axis='x', rotation=45)
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Cell 10: Statistical Insights and Key Findings
print("🔍 KEY FINDINGS AND STATISTICAL INSIGHTS")
print("=" * 50)

# Peak hour analysis
peak_hour = analysis_results['hourly'][('vehicle_count', 'mean')].idxmax()
peak_volume = analysis_results['hourly'][('vehicle_count', 'mean')].max()
print(f"📈 Peak Traffic Hour: {peak_hour}:00 ({peak_volume:.1f} vehicles)")

# Most congested road type
most_congested_road = analysis_results['road_type']['traffic_index'].idxmax()
congestion_level = analysis_results['road_type']['traffic_index'].max()
print(f"🚧 Most Congested Road Type: {most_congested_road} (Index: {congestion_level:.2f})")

# Weather impact
weather_impact = analysis_results['weather']['average_speed'].idxmin()
slowest_speed = analysis_results['weather']['average_speed'].min()
print(f"🌧️  Worst Weather for Traffic: {weather_impact} ({slowest_speed:.1f} km/h)")

# Efficiency analysis
most_efficient_road = analysis_results['road_type']['efficiency'].idxmax()
efficiency_score = analysis_results['road_type']['efficiency'].max()
print(f"✅ Most Efficient Road Type: {most_efficient_road} (Score: {efficiency_score:.2f})")

# Congestion distribution
congestion_dist = processed_data['congestion_level'].value_counts(normalize=True) * 100
print("\n🚦 Congestion Level Distribution:")
for level, percentage in congestion_dist.items():
    print(f"   {level}: {percentage:.1f}%")

In [ ]:
# Cell 11: Data Export and Save Results
print("💾 Exporting results...")

# Save processed data to CSV
processed_data.to_csv('traffic_analysis_results.csv', index=False)
print("✅ Data saved to 'traffic_analysis_results.csv'")

# Save summary statistics
summary_df = pd.DataFrame([summary_stats])
summary_df.to_csv('traffic_summary_statistics.csv', index=False)
print("✅ Summary statistics saved to 'traffic_summary_statistics.csv'")

# Save analysis results to Excel
with pd.ExcelWriter('traffic_analysis_report.xlsx') as writer:
    processed_data.to_excel(writer, sheet_name='Raw Data', index=False)
    analysis_results['hourly'].to_excel(writer, sheet_name='Hourly Analysis')
    analysis_results['daily'].to_excel(writer, sheet_name='Daily Analysis')
    analysis_results['road_type'].to_excel(writer, sheet_name='Road Type Analysis')
    analysis_results['weather'].to_excel(writer, sheet_name='Weather Impact')
print("✅ Comprehensive report saved to 'traffic_analysis_report.xlsx'")

# Display download links in notebook (for Jupyter environment)
from IPython.display import FileLinks
FileLinks('.', recursive=False)

In [ ]:
# Cell 12: Interactive Analysis (Optional - for Jupyter widgets)
try:
    from ipywidgets import interact, interactive, fixed, interact_manual
    import ipywidgets as widgets

    print("🎮 Interactive Analysis Controls")

    @interact
    def analyze_congestion_by_hour(hour=widgets.IntSlider(min=0, max=23, value=12, description='Hour:')):
        """Interactive analysis of congestion by hour."""
        hour_data = processed_data[processed_data['hour'] == hour]

        if len(hour_data) > 0:
            fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

            # Congestion distribution for selected hour
            congestion_counts = hour_data['congestion_level'].value_counts()
            ax1.pie(congestion_counts.values, labels=congestion_counts.index, autopct='%1.1f%%',
                   colors=['lightgreen', 'gold', 'orange', 'red'])
            ax1.set_title(f'Congestion Distribution at {hour}:00')

            # Average speed by road type for selected hour
            speed_by_road = hour_data.groupby('road_type')['average_speed'].mean()
            speed_by_road.plot(kind='bar', ax=ax2, color='skyblue', alpha=0.7)
            ax2.set_title(f'Average Speed by Road Type at {hour}:00')
            ax2.set_ylabel('Speed (km/h)')
            ax2.tick_params(axis='x', rotation=45)

            plt.tight_layout()
            plt.show()

            print(f"📊 Analysis for hour {hour}:00")
            print(f"   Total records: {len(hour_data)}")
            print(f"   Average speed: {hour_data['average_speed'].mean():.1f} km/h")
            print(f"   Average vehicle count: {hour_data['vehicle_count'].mean():.1f}")
        else:
            print(f"No data available for hour {hour}:00")

except ImportError:
    print("ℹ️  ipywidgets not available. Install with: pip install ipywidgets")

In [ ]:
# Cell 13: Final Summary and Recommendations
print("🎯 PROJECT SUMMARY AND RECOMMENDATIONS")
print("=" * 50)

print("\n📋 Project Overview:")
print("• Generated and analyzed synthetic traffic data for urban planning")
print("• Implemented complete data analysis pipeline")
print("• Created comprehensive visualizations and insights")
print("• Demonstrated GIS and data science techniques")

print("\n🚀 Technical Achievements:")
print("✓ Synthetic data generation with realistic patterns")
print("✓ Advanced feature engineering and calculations")
print("✓ Multi-faceted traffic pattern analysis")
print("✓ Interactive and static visualizations")
print("✓ Statistical insights and correlation analysis")
print("✓ Data export and reporting capabilities")

print("\n💡 Key Insights for Urban Planning:")
print("• Identify peak traffic hours for resource allocation")
print("• Optimize traffic light timing based on hourly patterns")
print("• Focus infrastructure improvements on most congested road types")
print("• Consider weather conditions in traffic management strategies")
print("• Monitor efficiency metrics for continuous improvement")

print("\n🔧 Libraries Used:")
libraries = {
    "numpy": "Numerical computations and array operations",
    "pandas": "Data manipulation and analysis",
    "matplotlib": "Static visualizations and plotting",
    "seaborn": "Statistical data visualization",
    "faker": "Synthetic data generation"
}

for lib, desc in libraries.items():
    print(f"• {lib}: {desc}")

print(f"\n✅ Analysis completed successfully!")
print(f"📁 Generated files: traffic_analysis_results.csv, traffic_summary_statistics.csv, traffic_analysis_report.xlsx")